In [451]:
import random
from itertools import permutations
import math

In [452]:
class Problem:
    """Esta es una clase abstracta para un problema.
    No se supone que debamos crear objetos de esta
    clase directamente. La idea es que otras clases
    hereden de esta para crear clases de problemas
    más especeificos."""
    
    def __init__(self, initial, goal=None):
        """Esta definición es un método, es especial
        porque comienza y termina con doble piso.
        En Python, el método __init__ es utilizado
        para construir objetos de la clase.
        En este y el resto de los métodos, el primer
        parámetro es self, y nos permite referirnos
        al objeto sobre el cuál el método es invocado."""
        self.initial = initial
        self.goal = goal
    
    def actions(self, state):
        """Un problema contiene un estado inicial y
        quizá una meta en particular. El método actions
        debe regresar las acciones que se pueden tomar
        en el estado dado. El resultado usualmente es
        una lista, pero si debes regresar muchas acciones,
        considera usar yield para generarlas una a la vez."""
        raise NotImplementedError
    
    def result(self, state, action):
        """El método anterior, al igual que este, tiene una
        implementación trivial. Simplemente señalamos un
        error que indica que no hay una implementación
        para el método. En una implementación de este método,
        debemos regresar el estado al que llegamos después
        de ejecutar la acción dada en el estado dado.
        La acción debe ser una calculada por
        self.actions(state)."""
        raise NotImplementedError

    def random_solution(self):
        raise NotImplementedError

    def neighbors_of(self, state):
        raise NotImplementedError
    
    def goal_test(self, state):
        """De seguro ya te diste cuenta que todos estos
        métodos son muy generales y poco específicos.
        ¿Qué forma tienen los estados?
        ¿Qué son las acciones?
        ¿Qué es una meta?
        Este método regresa True si el estado dado es una
        meta. Por defecto se compara el estado dado con
        self.goal o verifica que el estado dado sea
        elemento de self.goal en caso de ser lista."""
        if isinstance(self.goal, list):
            return any(x is state for x in self.goal)
        return state == self.goal
    
    
    def all_domain(self):
        return NotImplementedError
    
    def random_neighbor(self, state):
        raise NotImplementedError
    
    def crossover(self,state1,state2):
        raise NotImplementedError
    
    def value(self, state):
        """En distintos problemas, cada estado tiene un valor asociado.
        Este método sirve para obtener este valor."""
        raise NotImplementedError

In [453]:
def factorial(num):
        if num == 1:
            return num
        else:
            return factorial(num-1) * num

In [454]:
class NReinas(Problem):
    def __init__(self,n_col):
        self.n_col = n_col
        domain = factorial(self.n_col)
        self.domain = domain

    def random_solution(self):
        solution = list(range(self.n_col))
        random.shuffle(solution)
        return solution

    def value(self, state):
        diagonal1 = []
        diagonal2 = []
        for i in range(self.n_col):
            diagonal1.append(i+state[i])
            diagonal2.append(i-state[i])
        return 2*self.n_col - (len(set(diagonal1)) + len(set(diagonal2)))
    
    def neighbors_of(self,state):
        neighbors = []
        for i in range(len(state)):
            for j in range(i+1,len(state)):
                neighbors.append(state[:i] + state[j:j+1] + state [i+1:j] + state[i:i+1] + state[j+1:])
        return neighbors

    def domain_size(self):
        if self.domain > 10000000:
            return True
        else:
            return False
    
    def all_domain(self):
        all = [i for i in range(self.n_col)]
        return list(permutations(all))
    
    def crossover(self, s1, s2):
        i = random.randint(1, len(s1)-2)
        state2 = [elem for elem in s2 if elem not in s1[:i]]
        return s1[:i] + state2

    def random_neighbor(self, state):
        lista = [i for i in range(self.n_col)]
        x = lista.pop(random.randint(0,len(lista)-1))
        y = lista.pop(random.randint(0,len(lista)-1))
        if x < y:
            return state[0:x] + [state[y]] + state[x+1:y] + [state[x]] + state[y+1:]
        else:
            return state[0:y] + [state[x]] + state[y+1:x] + [state[y]] + state[x+1:]
        


In [455]:
class OptimazationMethod:
    def solve(slef, Problem):
        raise NotImplementedError

In [456]:
class ExhaustiveSearch(OptimazationMethod):
    def solve(self,Problem):
        if Problem.domain_size():
            return "Impossible to calculate with this optimization method in this computer."
        else:
            solution = []
            for element in Problem.all_domain():
                if Problem.value(element) == 0:
                    solution.append(element)
        return solution

In [457]:
class SolveRandomly(OptimazationMethod):
    def __init__(self, repeats: int = 1000):
        self.repeats = repeats
        
    def solve(self,Problem,):
        best_cost = float('inf')
        best_sol = None
    
        for _ in range(self.repeats):
            s = Problem.random_solution()
            c = Problem.value(s)
            if c <= best_cost:
                best_cost = c
                best_sol = s
    
        return best_sol, best_cost

In [458]:
class HillClimbing(OptimazationMethod):
    def solve(self,Problem):
        s = Problem.random_solution()

        while True:
            cost = Problem.value(s)
            neighbors = Problem.neighbors_of(s)
            best_neighbor = min(neighbors, key=Problem.value)
            neighbor_cost = Problem.value(best_neighbor)
            if cost <= neighbor_cost:
                return s, cost
            
            s = best_neighbor     

In [459]:
class Annealing(OptimazationMethod):
    def __init__(self,Ti=10000,Tf=.1,alpha=.95):
        self.Ti = Ti
        self.Tf = Tf
        self.alpha = alpha

    def solve(self,Problem):
        solution = Problem.random_solution()
        cost = Problem.value(solution)
        T=self.Ti
        while T > self.Tf:
            if Problem.neighbors_of(solution) == 0:
                return solution
            neighbor = Problem.random_neighbor(solution)
            neighbor_cost = Problem.value(neighbor)
            diff = cost - neighbor_cost
            if diff > 0 or random.random() < math.exp(diff / T):
                solution = neighbor
                cost = neighbor_cost
            T = self.alpha*T
        return solution, cost

In [460]:
class Evoling(OptimazationMethod):
    def __init__(self, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
        self.pop_size = pop_size
        self.mut_prob = mut_prob
        self.elite = elite
        self.epochs = epochs

    def solve(self, Problem):
        pop = [Problem.random_solution() for _ in range(self.pop_size)]
        top_elite = int(self.elite * self.pop_size)
        
        for epoch in range(self.epochs):
            pop.sort(key=Problem.value)
            best = pop[:top_elite]
            while len(best) < self.pop_size:
                if random.random() < self.mut_prob:
                    best.append(Problem.random_neighbor(
                        best[random.randint(0, top_elite-1)]
                    ))
                else:
                    best.append(Problem.crossover(
                        best[random.randint(0, top_elite-1)],
                        best[random.randint(0, top_elite-1)],
                    ))
            pop = best
        pop.sort(key=Problem.value)
        return pop[0], Problem.value(pop[0])

In [461]:
Reinas8 = NReinas(15)

In [462]:
print(ExhaustiveSearch().solve(Reinas8))

Impossible to calculate with this optimization method in this computer.


In [463]:
print(SolveRandomly().solve(Reinas8))

([11, 5, 10, 2, 4, 14, 7, 12, 8, 1, 13, 6, 3, 9, 0], 2)


In [484]:
print(HillClimbing().solve(Reinas8))

([1, 6, 9, 3, 13, 7, 12, 10, 0, 4, 14, 5, 8, 2, 11], 1)


In [465]:
print(Annealing().solve(Reinas8))

([7, 10, 3, 0, 5, 9, 8, 12, 2, 4, 13, 11, 14, 1, 6], 4)


In [495]:
print(Evoling().solve(Reinas8))

([13, 10, 1, 11, 5, 12, 4, 0, 8, 3, 14, 7, 9, 2, 6], 0)
